# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-4b20701de1-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Disney explorative search") 

Consider the following exploratory information need:

> investigate films produced by Disney and compare workers among films.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P106`     | profession   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q36479`    | The Lion King      | node |





Also consider

```
wd:Q36479 ?p ?obj .
```

is the BGP to retrieve all **properties of The Lion King**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for films

2. Identify the BGP for Walt Disney Animation Studios

3. Identify the BGP for Walt Disney (person)

4. List all the films created by the Walt Disney Animation Studios while Walt Disney was alive (the result set must be a list of couples films IRI and label).

5. Find the name of the voice actors who participated in more than 5 films by the Walt Disney Animation Studios (the result set must be a list of triples voice actors IRI, label and #films).

6. Identify the BGP for the film "The Little Mermaid"

7. Find who worked on both "The Little Mermaid" and "The Lion King" (the result set must be a list of couples worker IRI and label). 

8. Identify the BGP for Academy Awards

9. For each films created by the Walt Disney Animation Studios return the number of nomination for an Academy Awards. Return only films with at least one nomination. (the result set must be triples of films IRI, label and #nomination).

## Task 1
Identify the BGP for films

In [2]:
# retrieve all properties of The Lion King

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36479 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3092'), ('name', 'film crew member')]
[('p', 'http://www.wikidata.org/prop/direct/P4021'), ('name', 'danskefilm animated film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7679'), ('name', 'Cinépolis KLIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia B

In [33]:
# look at the directors of The Lion King
# wdt:P57 -- director

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36479 wdt:P57 ?p.
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1151111'), ('name', 'Rob Minkoff')]
[('p', 'http://www.wikidata.org/entity/Q847468'), ('name', 'Roger Allers')]
2


In [54]:
# look at the genre of The Lion King
# wdt:P136 -- genre

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36479 wdt:P136 ?p.
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q2143665'), ('name', "children's film")]
[('p', 'http://www.wikidata.org/entity/Q130232'), ('name', 'drama')]
[('p', 'http://www.wikidata.org/entity/Q842256'), ('name', 'musical film')]
[('p', 'http://www.wikidata.org/entity/Q28968511'), ('name', 'traditionally animated film')]
4


Final query for this task

In [61]:
# look at entities related to "children's film"
# [('p', 'http://www.wikidata.org/entity/Q2143665'), ('name', "children's film")]

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q2143665 ?b ?p.
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q201658'), ('name', 'film genre')]
[('p', 'http://www.wikidata.org/entity/Q6529327'), ('name', "Category:Children's films")]
[('p', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
[('p', 'http://www.wikidata.org/entity/Q1361932'), ('name', 'family film')]
[('p', 'http://www.wikidata.org/entity/Q56451354'), ('name', "children's fiction")]
[('p', 'http://www.wikidata.org/entity/Q5357245'), ('name', "list of children's films")]
6


## Task 2
Identify the BGP for Walt Disney Animation Studios

In [294]:
# using the [('p', 'http://www.wikidata.org/entity/Q1151111'), ('name', 'Rob Minkoff')] result, look up his properties

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1151111 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P1969'), ('name', 'MovieMeter director ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1971'), ('name', 'number of children')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('n

In [36]:
# get the employer of Rob Minkoff

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1151111 wdt:P108 ?p.
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q2216535'), ('name', 'Walt Disney Studios')]
1


Final query for this task

In [60]:
# look up Walt Disney Studios properties

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q2216535 ?b ?p.
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q375336'), ('name', 'film studio')]
[('p', 'http://www.wikidata.org/entity/Q4132994'), ('name', 'Disney Theatrical Group')]
[('p', 'http://www.wikidata.org/entity/Q1323594'), ('name', 'Walt Disney Studios Motion Pictures')]
[('p', 'http://www.wikidata.org/entity/Q191224'), ('name', 'Walt Disney Pictures')]
[('p', 'http://www.wikidata.org/entity/Q7414'), ('name', 'The Walt Disney Company')]
[('p', 'http://www.wikidata.org/entity/Q367466'), ('name', 'Marvel Studios')]
[('p', 'http://www.wikidata.org/entity/Q885885'), ('name', 'Blue Sky Studios')]
[('p', 'http://www.wikidata.org/entity/Q953040'), ('name', 'Searchlight Pictures')]
[('p', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
[('p', 'http://www.wikidata.org/entity/Q2104471'), ('name', 'Disney Music Group')]
[('p', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('p', 'http://www.wikidata.org/entity/Q127552'), ('name', 'Pixar')]
[('p', 'http://

## Task 3
Identify the BGP for Walt Disney (person)

Final query for this task

In [59]:
# from the previous query we can obtain entities related to 'Walt Disney Animation Studios'
# [('p', 'http://www.wikidata.org/entity/Q1047410'), ('name', 'Walt Disney Animation Studios')]

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1047410 ?b ?p.
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q8610'), ('name', 'Academy Scientific and Technical Award')]
[('p', 'http://www.wikidata.org/entity/Q191224'), ('name', 'Walt Disney Pictures')]
[('p', 'http://www.wikidata.org/entity/Q7414'), ('name', 'The Walt Disney Company')]
[('p', 'http://www.wikidata.org/entity/Q1762059'), ('name', 'film production company')]
[('p', 'http://www.wikidata.org/entity/Q5127380'), ('name', 'Clark Spencer')]
[('p', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('p', 'http://www.wikidata.org/entity/Q4819492'), ('name', 'Auckland Art Gallery')]
[('p', 'http://www.wikidata.org/entity/Q891723'), ('name', 'public company')]
[('p', 'http://www.wikidata.org/entity/Q202866'), ('name', 'animated film')]
[('p', 'http://www.wikidata.org/entity/Q1415395'), ('name', 'film industry')]
[('p', 'http://www.wikidata.org/entity/Q2620855'), ('name', 'Satellite Award for Best Animated or Mixed Media Feature')]
[('p', 'http://www.wikidata.org/en

## Task 4
List all the films created by the Walt Disney Animation Studios while Walt Disney was alive (the result set must be a list of couples films IRI and label).

In [145]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1047410 ?a ?p.
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q8610'), ('name', 'Academy Scientific and Technical Award')]
[('p', 'http://www.wikidata.org/entity/Q191224'), ('name', 'Walt Disney Pictures')]
[('p', 'http://www.wikidata.org/entity/Q7414'), ('name', 'The Walt Disney Company')]
[('p', 'http://www.wikidata.org/entity/Q1762059'), ('name', 'film production company')]
[('p', 'http://www.wikidata.org/entity/Q5127380'), ('name', 'Clark Spencer')]
[('p', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('p', 'http://www.wikidata.org/entity/Q4819492'), ('name', 'Auckland Art Gallery')]
[('p', 'http://www.wikidata.org/entity/Q891723'), ('name', 'public company')]
[('p', 'http://www.wikidata.org/entity/Q202866'), ('name', 'animated film')]
[('p', 'http://www.wikidata.org/entity/Q1415395'), ('name', 'film industry')]
[('p', 'http://www.wikidata.org/entity/Q2620855'), ('name', 'Satellite Award for Best Animated or Mixed Media Feature')]
[('p', 'http://www.wikidata.org/en

In [149]:
#look for a property to get the date of death for a human

queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
   wd:Q5 ?a ?p.
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/P1317'), ('name', 'floruit')]
[('p', 'http://www.wikidata.org/entity/P1477'), ('name', 'birth name')]
[('p', 'http://www.wikidata.org/entity/P1559'), ('name', 'name in native language')]
[('p', 'http://www.wikidata.org/entity/P509'), ('name', 'cause of death')]
[('p', 'http://www.wikidata.org/entity/P3373'), ('name', 'sibling')]
[('p', 'http://www.wikidata.org/entity/Q785260'), ('name', 'phase of human life')]
[('p', 'http://www.wikidata.org/entity/Q95074'), ('name', 'fictional character')]
[('p', 'http://www.wikidata.org/entity/Q23404'), ('name', 'anthropology')]
[('p', 'http://www.wikidata.org/entity/Q3769299'), ('name', 'human behavior')]
[('p', 'http://www.wikidata.org/entity/Q8425'), ('name', 'society')]
[('p', 'http://www.wikidata.org/entity/P119'), ('name', 'place of burial')]
[('p', 'http://www.wikidata.org/entity/Q215627'), ('name', 'person')]
[('p', 'http://www.wikidata.org/entity/Q1071027'), ('name', 'personal name')]
[('p', 'ht

In [169]:
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
   wd:Q3072145 ?obj ?p.
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
[('p', 'http://www.wikidata.org/entity/Q8704'), ('name', 'Walt Disney')]
[('p', 'http://www.wikidata.org/entity/Q1371849'), ('name', 'filmography')]
3


In [75]:
#retrieve date of death (wdt:P570) for Walt Disney (wd:Q8704)

queryString = """
SELECT DISTINCT YEAR(?deathYear) as ?death ?person ?name
WHERE {
   # bind something
   VALUES ?person {wd:Q8704}
   wd:Q8704 wdt:P570 ?deathYear.
   # get the label
   ?death sc:name ?date.
   ?person sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('death', '1966'), ('person', 'http://www.wikidata.org/entity/Q8704'), ('name', 'Walt Disney')]
1


In [376]:
#look if there's a directed by property for film
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
   wd:Q11424 ?a ?p.
   # get the label
   ?p sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/entity/Q1660187'), ('name', 'film studies')]
[('p', 'http://www.wikidata.org/entity/P345'), ('name', 'IMDb ID')]
[('p', 'http://www.wikidata.org/entity/P1040'), ('name', 'film editor')]
[('p', 'http://www.wikidata.org/entity/P344'), ('name', 'director of photography')]
[('p', 'http://www.wikidata.org/entity/Q17738'), ('name', 'Star Wars: Episode IV – A New Hope')]
[('p', 'http://www.wikidata.org/entity/Q55321'), ('name', 'Category:Films')]
[('p', 'http://www.wikidata.org/entity/Q20937557'), ('name', 'series')]
[('p', 'http://www.wikidata.org/entity/P1476'), ('name', 'title')]
[('p', 'http://www.wikidata.org/entity/P2047'), ('name', 'duration')]
[('p', 'http://www.wikidata.org/entity/P162'), ('name', 'producer')]
10


In [ ]:
# Q11424 -- 'film'
# [('p', 'http://www.wikidata.org/entity/P577'), ('name', 'publication date')]
# [('p', 'http://www.wikidata.org/entity/P570'), ('name', 'date of death')]

In [175]:
#look if there's a directed by property for film

queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
   wd:Q1047410 ?a ?p.
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q8610'), ('name', 'Academy Scientific and Technical Award')]
[('p', 'http://www.wikidata.org/entity/Q191224'), ('name', 'Walt Disney Pictures')]
[('p', 'http://www.wikidata.org/entity/Q7414'), ('name', 'The Walt Disney Company')]
[('p', 'http://www.wikidata.org/entity/Q1762059'), ('name', 'film production company')]
[('p', 'http://www.wikidata.org/entity/Q5127380'), ('name', 'Clark Spencer')]
[('p', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('p', 'http://www.wikidata.org/entity/Q4819492'), ('name', 'Auckland Art Gallery')]
[('p', 'http://www.wikidata.org/entity/Q891723'), ('name', 'public company')]
[('p', 'http://www.wikidata.org/entity/Q202866'), ('name', 'animated film')]
[('p', 'http://www.wikidata.org/entity/Q1415395'), ('name', 'film industry')]
[('p', 'http://www.wikidata.org/entity/Q2620855'), ('name', 'Satellite Award for Best Animated or Mixed Media Feature')]
[('p', 'http://www.wikidata.org/en

In [379]:
#looking up Walk Disney occupations

queryString = """
SELECT DISTINCT ?obj ?name 
WHERE {
   # bind something
   wd:Q8704 wdt:P106 ?obj.
   # get the label
   ?obj sc:name ?name.
}
LIMIT 15
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q3658608'), ('name', 'caricaturist')]
[('obj', 'http://www.wikidata.org/entity/Q1028181'), ('name', 'painter')]
[('obj', 'http://www.wikidata.org/entity/Q10800557'), ('name', 'film actor')]
[('obj', 'http://www.wikidata.org/entity/Q13235160'), ('name', 'manufacturer')]
[('obj', 'http://www.wikidata.org/entity/Q205375'), ('name', 'inventor')]
[('obj', 'http://www.wikidata.org/entity/Q2405480'), ('name', 'voice actor')]
[('obj', 'http://www.wikidata.org/entity/Q2526255'), ('name', 'film director')]
[('obj', 'http://www.wikidata.org/entity/Q28389'), ('name', 'screenwriter')]
[('obj', 'http://www.wikidata.org/entity/Q3282637'), ('name', 'film producer')]
[('obj', 'http://www.wikidata.org/entity/Q33999'), ('name', 'actor')]
[('obj', 'http://www.wikidata.org/entity/Q3455803'), ('name', 'director')]
[('obj', 'http://www.wikidata.org/entity/Q36180'), ('name', 'writer')]
[('obj', 'http://www.wikidata.org/entity/Q644687'), ('name', 'illustrator')]

In [186]:
# checking if the film has a 'publication date' property

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36479 ?p ?a.
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3092'), ('name', 'film crew member')]
[('p', 'http://www.wikidata.org/prop/direct/P4021'), ('name', 'danskefilm animated film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7679'), ('name', 'Cinépolis KLIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia B

In [76]:
# test
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36479 ?p wd:Q191224.
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('p', 'http://www.wikidata.org/prop/direct/P272'), ('name', 'production company')]
1


In [96]:
# films directed by walt disney animation studios

queryString = """
SELECT DISTINCT ?film ?name 
WHERE {
   # bind something
   ?film wdt:P272 wd:Q1047410.
   # get the label
   ?film sc:name ?name.
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('film', 'http://www.wikidata.org/entity/Q223163'), ('name', "The Emperor's New Groove")]
[('film', 'http://www.wikidata.org/entity/Q18647981'), ('name', 'Moana')]
[('film', 'http://www.wikidata.org/entity/Q179673'), ('name', 'Beauty and the Beast')]
[('film', 'http://www.wikidata.org/entity/Q15046569'), ('name', 'Roger Rabbit short films')]
[('film', 'http://www.wikidata.org/entity/Q3362144'), ('name', 'Trail Mix-Up')]
[('film', 'http://www.wikidata.org/entity/Q379873'), ('name', 'Who Framed Roger Rabbit')]
[('film', 'http://www.wikidata.org/entity/Q36479'), ('name', 'The Lion King')]
[('film', 'http://www.wikidata.org/entity/Q215617'), ('name', 'Sleeping Beauty')]
[('film', 'http://www.wikidata.org/entity/Q212792'), ('name', 'Bolt')]
[('film', 'http://www.wikidata.org/entity/Q944287'), ('name', 'The Little Mermaid')]
[('film', 'http://www.wikidata.org/entity/Q3236849'), ('name', "Let's Stick Together")]
[('film', 'http://www.wikidata.org/entity/Q25167044'), ('name', 'Ralph B

In [91]:
# look at 'publication date' for specific film

queryString = """
SELECT DISTINCT YEAR(?date) as ?publication ?film ?filmName
WHERE {
   # bind something
   VALUES ?film {wd:Q246283}
   ?film wdt:P577 ?date.
   # get the label
   ?publication sc:name ?datePub.
   ?film sc:name ?filmName.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('publication', '2013'), ('film', 'http://www.wikidata.org/entity/Q246283'), ('filmName', 'Frozen')]
[('publication', '2014'), ('film', 'http://www.wikidata.org/entity/Q246283'), ('filmName', 'Frozen')]
2


In [81]:
# checking the wikidata for Frozen we can see that has multiple publication date so the query is correct despite having 2 different year as result

Final query for this task

In [122]:
# List all the films created by the Walt Disney Animation Studios while Walt Disney was alive (the result set must be a list of couples films IRI and label).

queryString = """
SELECT DISTINCT ?film ?name
WHERE {
   # bind something
   VALUES ?person {wd:Q8704}
   wd:Q8704 wdt:P570 ?deathYear.
   ?film wdt:P272 wd:Q1047410;
         wdt:P577 ?datePub.
   #?death sc:name ?date.
   #?publication sc:name ?datePub.
   ?film sc:name ?name.
   FILTER(?datePub < ?deathYear).
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q215617'), ('name', 'Sleeping Beauty')]
[('film', 'http://www.wikidata.org/entity/Q3236849'), ('name', "Let's Stick Together")]
[('film', 'http://www.wikidata.org/entity/Q842306'), ('name', 'Saludos Amigos')]
[('film', 'http://www.wikidata.org/entity/Q2090328'), ('name', 'Trick or Treat')]
[('film', 'http://www.wikidata.org/entity/Q640679'), ('name', 'Rugged Bear')]
[('film', 'http://www.wikidata.org/entity/Q2399982'), ('name', 'Tea for Two Hundred')]
[('film', 'http://www.wikidata.org/entity/Q3028329'), ('name', 'Two Weeks Vacation')]
[('film', 'http://www.wikidata.org/entity/Q3520106'), ('name', 'The Brave Engineer')]
[('film', 'http://www.wikidata.org/entity/Q2700743'), ('name', 'Clock Cleaners')]
[('film', 'http://www.wikidata.org/entity/Q3521221'), ('name', 'The Haunted House')]
[('film', 'http://www.wikidata.org/entity/Q3036174'), ('name', 'Grin and Bear It')]
[('film', 'http://www.wikidata.org/entity/Q3228508'), ('name', 'Hold Th

In [ ]:
# if we want to return also date of publication we need to add:
# YEAR(?datePub) as ?publication
# in the SELECT form

## Task 5
Find the name of the voice actors who participated in more than 5 films by the Walt Disney Animation Studios (the result set must be a list of triples voice actors IRI, label and #films).

In [41]:
# [('p', 'http://www.wikidata.org/prop/direct/P3092'), ('name', 'film crew member')] 

queryString = """
SELECT DISTINCT ?actor ?name 
WHERE {
   # bind something
   wd:Q36479 wdt:P3092 ?actor.
   # get the label
   ?actor sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('actor', 'http://www.wikidata.org/entity/Q16204825'), ('name', 'Barry Kooser')]
[('actor', 'http://www.wikidata.org/entity/Q19822599'), ('name', 'Robert Walker')]
[('actor', 'http://www.wikidata.org/entity/Q28803503'), ('name', 'Robert Stanton')]
[('actor', 'http://www.wikidata.org/entity/Q28813470'), ('name', 'Vera Pacheco')]
[('actor', 'http://www.wikidata.org/entity/Q28855009'), ('name', 'Ric Sluiter')]
[('actor', 'http://www.wikidata.org/entity/Q28859357'), ('name', 'Jeff Dickson')]
[('actor', 'http://www.wikidata.org/entity/Q28859686'), ('name', 'Jeff Dutton')]
[('actor', 'http://www.wikidata.org/entity/Q28861733'), ('name', 'Tom Humber')]
[('actor', 'http://www.wikidata.org/entity/Q28861925'), ('name', 'Charles R. Vollmer')]
[('actor', 'http://www.wikidata.org/entity/Q28866085'), ('name', 'Daniel A. Gracey')]
10


In [238]:
# get the entities related to a specific film crew member

queryString = """
SELECT DISTINCT ?date ?name 
WHERE {
   # bind something
   wd:Q28813470 ?p ?date.
   # get the label
   ?date sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('date', 'http://www.wikidata.org/entity/Q406181'), ('name', 'Pacheco')]
[('date', 'http://www.wikidata.org/entity/Q5'), ('name', 'human')]
[('date', 'http://www.wikidata.org/entity/Q6581072'), ('name', 'female')]
[('date', 'http://www.wikidata.org/entity/Q28813302'), ('name', 'clean-up artist')]
[('date', 'http://www.wikidata.org/entity/Q659189'), ('name', 'Vera')]
[('date', 'http://www.wikidata.org/entity/Q28813489'), ('name', 'Lanpher')]
6


In [269]:
#voice actor isn't listed in film crew member

queryString = """
SELECT DISTINCT ?crew ?name ?job
WHERE {
   # bind something
   wd:Q18647981 wdt:P3092 ?crew.
   ?crew wdt:P106 ?job.
   # get the label
   ?job sc:name ?name.
   
   
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('crew', 'http://www.wikidata.org/entity/Q27915886'), ('name', 'character animator'), ('job', 'http://www.wikidata.org/entity/Q28122926')]
[('crew', 'http://www.wikidata.org/entity/Q10307886'), ('name', 'character animator'), ('job', 'http://www.wikidata.org/entity/Q28122926')]
[('crew', 'http://www.wikidata.org/entity/Q27915867'), ('name', 'visual effects supervisor'), ('job', 'http://www.wikidata.org/entity/Q8311')]
[('crew', 'http://www.wikidata.org/entity/Q28007190'), ('name', 'casting director'), ('job', 'http://www.wikidata.org/entity/Q1049296')]
[('crew', 'http://www.wikidata.org/entity/Q27916569'), ('name', 'special effects artist'), ('job', 'http://www.wikidata.org/entity/Q21560152')]
[('crew', 'http://www.wikidata.org/entity/Q2964799'), ('name', 'film producer'), ('job', 'http://www.wikidata.org/entity/Q3282637')]
[('crew', 'http://www.wikidata.org/entity/Q27915856'), ('name', 'film producer'), ('job', 'http://www.wikidata.org/entity/Q3282637')]
[('crew', 'http://www

In [270]:
queryString = """
SELECT DISTINCT ?job ?name 
WHERE {
   # bind something
   ?p wdt:P106 ?job;
   wdt:P108 
   # get the label
   ?job sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('job', 'http://www.wikidata.org/entity/Q28122926'), ('name', 'character animator')]
[('job', 'http://www.wikidata.org/entity/Q2707485'), ('name', 'scenographer')]
[('job', 'http://www.wikidata.org/entity/Q2962070'), ('name', 'production designer')]
[('job', 'http://www.wikidata.org/entity/Q1062952'), ('name', 'character designer')]
[('job', 'http://www.wikidata.org/entity/Q1643514'), ('name', 'music arranger')]
[('job', 'http://www.wikidata.org/entity/Q706364'), ('name', 'art director')]
[('job', 'http://www.wikidata.org/entity/Q1153051'), ('name', 'inker')]
[('job', 'http://www.wikidata.org/entity/Q21560152'), ('name', 'special effects artist')]
[('job', 'http://www.wikidata.org/entity/Q1028181'), ('name', 'painter')]
[('job', 'http://www.wikidata.org/entity/Q13235160'), ('name', 'manufacturer')]
10


In [292]:
# Q2405480 -- voice actor 
# Q1047410 -- walt disney animation studio
# Q8704 -- walt disney
# Q2216535 -- walt disney productions

queryString = """
SELECT DISTINCT ?job ?name 
WHERE {
   # bind something
   ?person wdt:P106 wd:Q2405480;
           wdt:P108 wd:Q2216535.
   # get the label
   ?person sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Jordy Ranft')]
[('name', 'Tony Bancroft')]
[('name', 'Bill Melendez')]
[('name', 'Chris Sanders')]
[('name', 'Eric Goldberg')]
[('name', 'Brad Bird')]
[('name', 'Tony Anselmo')]
[('name', 'Byron Howard')]
[('name', 'Joe Ranft')]
[('name', 'Gary Trousdale')]
[('name', 'Will Finn')]
[('name', 'Cal Howard')]
12


In [298]:
# Q2405480 -- voice actor 
# Q1047410 -- walt disney animation studio
# Q8704 -- walt disney
# Q2216535 -- walt disney productions
# P6181
queryString = """
SELECT DISTINCT ?job ?name 
WHERE {
   # bind something
   ?person wdt:P106 wd:Q2405480;
           wdt:P108 wd:Q1047410.
   # get the label
   ?person sc:name ?name.
}
LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Chris Williams')]
[('name', 'Paul Briggs')]
[('name', 'Fabienne Rawley')]
[('name', 'Lindsey Collins')]
4


In [300]:
# Q2405480 -- voice actor 
# Q1047410 -- walt disney animation studio
# Q8704
# Q2216535
# P6181 -- has disney ID
queryString = """
SELECT DISTINCT ?job ?name 
WHERE {
   # bind something
   ?person wdt:P106 wd:Q2405480;
           wdt:P6181 ?p.
   # get the label
   ?person sc:name ?name.
}
LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Bill Shirley')]
[('name', 'Stuart Buchanan')]
[('name', 'Bill Mumy')]
[('name', 'Clint Howard')]
[('name', 'Dean Jones')]
[('name', 'Sally Field')]
[('name', 'Robert Hays')]
[('name', 'Kathleen Turner')]
[('name', 'Carl Reiner')]
[('name', 'Jonathan Brandis')]
[('name', 'David Paymer')]
[('name', 'Julia Roberts')]
[('name', 'Reese Witherspoon')]
[('name', 'Fred Willard')]
[('name', 'Powers Boothe')]
[('name', 'Alfre Woodard')]
[('name', 'Leslie Nielsen')]
[('name', 'Hayley Atwell')]
[('name', 'Ming-Na Wen')]
[('name', 'Lea Salonga')]
[('name', 'Patrick McGoohan')]
[('name', 'Tony Bancroft')]
[('name', 'Eddie Murphy')]
[('name', 'Rex Allen')]
[('name', 'Christian Bale')]
[('name', 'Richard White')]
[('name', 'George Dzundza')]
[('name', 'Janeane Garofalo')]
[('name', 'Doug E. Doug')]
[('name', 'Bob Newhart')]
[('name', 'Chevy Chase')]
[('name', 'Margaret Kerry')]
[('name', 'Jonathan Winters')]
[('name', 'Cliff Edwards')]
[('name', 'Candy Candido')]
[('name', 'Jennifer

In [321]:
#look at properties of a generic walt disney animation studio product
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
   wd:Q944287 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('name', 'creator')]
[('p', 'http://www.wikidata.org/prop/direct/P1811'), ('name', 'list of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P2047'), ('name', 'duration')]
[('p', 'http://www.wikidata.org/prop/direct/P2364'), ('name', 'production code')]
10


In [333]:
# films directed by walt disney animation studios and their voice actor
queryString = """
SELECT DISTINCT ?person ?name ?film
WHERE {
   # bind something
   ?person wdt:P106 wd:Q2405480.
   ?film wdt:P272 wd:Q1047410;
         wdt:P6181 ?p;
         wdt:P161 ?person.
 
   # get the label
   ?person sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q2607618'), ('name', 'Mark Walton'), ('film', 'http://www.wikidata.org/entity/Q3977597')]
[('person', 'http://www.wikidata.org/entity/Q40220'), ('name', 'Susie Essman'), ('film', 'http://www.wikidata.org/entity/Q3977597')]
[('person', 'http://www.wikidata.org/entity/Q1138602'), ('name', 'Jodi Benson'), ('film', 'http://www.wikidata.org/entity/Q944287')]
[('person', 'http://www.wikidata.org/entity/Q3082635'), ('name', 'Frank Graham'), ('film', 'http://www.wikidata.org/entity/Q842306')]
[('person', 'http://www.wikidata.org/entity/Q4235'), ('name', 'Miley Cyrus'), ('film', 'http://www.wikidata.org/entity/Q3977597')]
[('person', 'http://www.wikidata.org/entity/Q109324'), ('name', 'Christopher Lloyd'), ('film', 'http://www.wikidata.org/entity/Q379873')]
[('person', 'http://www.wikidata.org/entity/Q117392'), ('name', 'Malcolm McDowell'), ('film', 'http://www.wikidata.org/entity/Q3977597')]
[('person', 'http://www.wikidata.org/entity/Q8704')

In [ ]:
#when trying to look up voice actors who worked in film produced by walt disney animation studios we get too few results

In [326]:
# when searching for voice actors who works in disney I found different result using different properties, such as: employed by and has Disney ID.
# the most results (499) were obtained using disney ID so I will use that property to complete the task.

In [387]:
queryString = """
SELECT DISTINCT ?job ?name ?person
WHERE {
   # bind something
   ?person wdt:P106 wd:Q2405480;
           wdt:P6181 ?p.
   # get the label
   ?person sc:name ?name.
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Bill Shirley'), ('person', 'http://www.wikidata.org/entity/Q2903350')]
[('name', 'Stuart Buchanan'), ('person', 'http://www.wikidata.org/entity/Q3500782')]
[('name', 'Bill Mumy'), ('person', 'http://www.wikidata.org/entity/Q748584')]
[('name', 'Clint Howard'), ('person', 'http://www.wikidata.org/entity/Q1101612')]
[('name', 'Dean Jones'), ('person', 'http://www.wikidata.org/entity/Q1181089')]
[('name', 'Sally Field'), ('person', 'http://www.wikidata.org/entity/Q187033')]
[('name', 'Robert Hays'), ('person', 'http://www.wikidata.org/entity/Q1970920')]
[('name', 'Kathleen Turner'), ('person', 'http://www.wikidata.org/entity/Q222008')]
[('name', 'Carl Reiner'), ('person', 'http://www.wikidata.org/entity/Q287793')]
[('name', 'Jonathan Brandis'), ('person', 'http://www.wikidata.org/entity/Q317539')]
[('name', 'David Paymer'), ('person', 'http://www.wikidata.org/entity/Q373989')]
[('name', 'Julia Roberts'), ('person', 'http://www.wikidata.org/entity/Q40523')]
[('name', 'Re

In [124]:
#executing the query using the property film directed by Walt Disney Animation studios we get the following results which seem to be incomplete due to the few data.

queryString = """
SELECT DISTINCT ?name ?person (COUNT (?film) as ?films)
WHERE {
   # bind something
   ?person wdt:P106 wd:Q2405480.
   ?film wdt:P272 wd:Q1047410;
         wdt:P161 ?person.
   # get the label
   ?person sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Alfre Woodard'), ('person', 'http://www.wikidata.org/entity/Q526620'), ('films', '1')]
[('name', 'Amy Adams'), ('person', 'http://www.wikidata.org/entity/Q481832'), ('films', '1')]
[('name', 'Malcolm McDowell'), ('person', 'http://www.wikidata.org/entity/Q117392'), ('films', '1')]
[('name', 'James Earl Jones'), ('person', 'http://www.wikidata.org/entity/Q203960'), ('films', '1')]
[('name', 'Joanna Cassidy'), ('person', 'http://www.wikidata.org/entity/Q236702'), ('films', '1')]
[('name', 'Richard LeParmentier'), ('person', 'http://www.wikidata.org/entity/Q367085'), ('films', '1')]
[('name', 'Susan Sarandon'), ('person', 'http://www.wikidata.org/entity/Q133050'), ('films', '1')]
[('name', 'Jodi Benson'), ('person', 'http://www.wikidata.org/entity/Q1138602'), ('films', '2')]
[('name', 'Miley Cyrus'), ('person', 'http://www.wikidata.org/entity/Q4235'), ('films', '1')]
[('name', 'John Fiedler'), ('person', 'http://www.wikidata.org/entity/Q944638'), ('films', '1')]
[('name

In [123]:
#executing the query using the property Disney ID associated at the actor we get the following results but the query counts also films where the actor isn't working for Walt Disney animated studio

queryString = """
SELECT DISTINCT ?name ?person (COUNT (?film) as ?films)
WHERE {
   # bind something
   ?person wdt:P106 wd:Q2405480;
           wdt:P6181 ?p.
   ?film wdt:P161 ?person.
   # get the label
   ?person sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Bill Mumy'), ('person', 'http://www.wikidata.org/entity/Q748584'), ('films', '15')]
[('name', 'Julia Roberts'), ('person', 'http://www.wikidata.org/entity/Q40523'), ('films', '51')]
[('name', 'Reese Witherspoon'), ('person', 'http://www.wikidata.org/entity/Q44063'), ('films', '43')]
[('name', 'Alfre Woodard'), ('person', 'http://www.wikidata.org/entity/Q526620'), ('films', '84')]
[('name', 'Samuel L. Jackson'), ('person', 'http://www.wikidata.org/entity/Q172678'), ('films', '145')]
[('name', 'Tate Donovan'), ('person', 'http://www.wikidata.org/entity/Q302650'), ('films', '52')]
[('name', 'Nancy Travis'), ('person', 'http://www.wikidata.org/entity/Q233042'), ('films', '31')]
[('name', 'Margot Kidder'), ('person', 'http://www.wikidata.org/entity/Q234471'), ('films', '53')]
[('name', 'Susan Egan'), ('person', 'http://www.wikidata.org/entity/Q3978226'), ('films', '11')]
[('name', 'Anjelica Huston'), ('person', 'http://www.wikidata.org/entity/Q190998'), ('films', '63')]
[

In [401]:
# This method could not be 100% accurate because there might be films that have a Disney ID but were given to a different animation studio.
# However it's the only way to obtain a query result with the data given.

In [125]:
#executing the query asking for a Disney ID for both voice actors and film, this way we overcome the lack of data for 'films directed by Walt Disney animation studio'. 

queryString = """
SELECT DISTINCT ?name ?person (COUNT (?film) as ?films)
WHERE {
   # bind something
   ?person wdt:P106 wd:Q2405480;
           wdt:P6181 ?p.
   ?film wdt:P161 ?person;
         wdt:P6181 ?g.
   # get the label
   ?person sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Corbin Bleu'), ('person', 'http://www.wikidata.org/entity/Q4617'), ('films', '6')]
[('name', 'America Ferrera'), ('person', 'http://www.wikidata.org/entity/Q219402'), ('films', '2')]
[('name', 'Vin Diesel'), ('person', 'http://www.wikidata.org/entity/Q178166'), ('films', '2')]
[('name', 'Jesse McCartney'), ('person', 'http://www.wikidata.org/entity/Q191842'), ('films', '2')]
[('name', 'John Cusack'), ('person', 'http://www.wikidata.org/entity/Q106175'), ('films', '6')]
[('name', 'Jim Belushi'), ('person', 'http://www.wikidata.org/entity/Q107933'), ('films', '7')]
[('name', 'Shia LaBeouf'), ('person', 'http://www.wikidata.org/entity/Q180942'), ('films', '6')]
[('name', 'Billy Crystal'), ('person', 'http://www.wikidata.org/entity/Q186485'), ('films', '2')]
[('name', 'Frank Welker'), ('person', 'http://www.wikidata.org/entity/Q296577'), ('films', '9')]
[('name', 'Jack Albertson'), ('person', 'http://www.wikidata.org/entity/Q344735'), ('films', '3')]
[('name', 'Alicia Wi

Final query for this task

In [167]:
# the last step is to filter voice actors that have worked in less than 5 movies

queryString = """
SELECT DISTINCT ?name ?person (COUNT (?film) AS ?filmsVoiced)
WHERE {
   # bind something
   ?person wdt:P106 wd:Q2405480;
           wdt:P6181 ?p.
   ?film wdt:P161 ?person;
         wdt:P6181 ?g.
   # get the label
   ?person sc:name ?name.
   #FILTER (?filmsVoiced > 5)
}
GROUP BY ?name ?person
#HAVING (?filmsVoiced > 5)
#ORDER BY DESC (?filmsVoiced)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Dan Aykroyd'), ('person', 'http://www.wikidata.org/entity/Q105221'), ('filmsVoiced', '5')]
[('name', 'Vin Diesel'), ('person', 'http://www.wikidata.org/entity/Q178166'), ('filmsVoiced', '2')]
[('name', 'Ed Asner'), ('person', 'http://www.wikidata.org/entity/Q154421'), ('filmsVoiced', '2')]
[('name', 'Jim Belushi'), ('person', 'http://www.wikidata.org/entity/Q107933'), ('filmsVoiced', '7')]
[('name', 'James Coburn'), ('person', 'http://www.wikidata.org/entity/Q4227'), ('filmsVoiced', '2')]
[('name', 'Geraldine Page'), ('person', 'http://www.wikidata.org/entity/Q228650'), ('filmsVoiced', '1')]
[('name', 'Helen Slater'), ('person', 'http://www.wikidata.org/entity/Q241879'), ('filmsVoiced', '2')]
[('name', 'Joe Ranft'), ('person', 'http://www.wikidata.org/entity/Q453094'), ('filmsVoiced', '2')]
[('name', 'Donald Sutherland'), ('person', 'http://www.wikidata.org/entity/Q103784'), ('filmsVoiced', '3')]
[('name', 'Teri Hatcher'), ('person', 'http://www.wikidata.org/entity/Q

## Task 6
Identify the BGP for the film "The Little Mermaid"

Final query for this task

In [373]:
# films directed by walt disney animation studios
queryString = """
SELECT DISTINCT ?film ?name ?director
WHERE {
   # bind something
   ?film wdt:P272 wd:Q1047410.
   # get the label
   ?film sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q223163'), ('name', "The Emperor's New Groove")]
[('film', 'http://www.wikidata.org/entity/Q18647981'), ('name', 'Moana')]
[('film', 'http://www.wikidata.org/entity/Q179673'), ('name', 'Beauty and the Beast')]
[('film', 'http://www.wikidata.org/entity/Q15046569'), ('name', 'Roger Rabbit short films')]
[('film', 'http://www.wikidata.org/entity/Q3362144'), ('name', 'Trail Mix-Up')]
[('film', 'http://www.wikidata.org/entity/Q379873'), ('name', 'Who Framed Roger Rabbit')]
[('film', 'http://www.wikidata.org/entity/Q36479'), ('name', 'The Lion King')]
[('film', 'http://www.wikidata.org/entity/Q215617'), ('name', 'Sleeping Beauty')]
[('film', 'http://www.wikidata.org/entity/Q212792'), ('name', 'Bolt')]
[('film', 'http://www.wikidata.org/entity/Q944287'), ('name', 'The Little Mermaid')]
10


In [ ]:
#[('film', 'http://www.wikidata.org/entity/Q944287'), ('name', 'The Little Mermaid')]

## Task 7
Find who worked on both "The Little Mermaid" and "The Lion King" (the result set must be a list of couples worker IRI and label).

In [365]:
# "The Little Mermaid" datas
# wd:Q944287 -- The Little Mermaid

queryString = """
SELECT DISTINCT ?person ?name ?film
WHERE {
   # bind something
   wd:Q944287 ?p ?person.
   # get the label
   ?person sc:name ?name.
   
}
LIMIT 25
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q3565642'), ('name', 'Walt Disney Television')]
[('person', 'http://www.wikidata.org/entity/Q3015230'), ('name', 'Danny Cooksey')]
[('person', 'http://www.wikidata.org/entity/Q43380'), ('name', 'CBS')]
[('person', 'http://www.wikidata.org/entity/Q1138602'), ('name', 'Jodi Benson')]
[('person', 'http://www.wikidata.org/entity/Q1264306'), ('name', 'Pat Carroll')]
[('person', 'http://www.wikidata.org/entity/Q190135'), ('name', 'The Little Mermaid')]
[('person', 'http://www.wikidata.org/entity/Q926362'), ('name', 'Samuel E. Wright')]
[('person', 'http://www.wikidata.org/entity/Q1860'), ('name', 'English')]
[('person', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('person', 'http://www.wikidata.org/entity/Q5398426'), ('name', 'television series')]
[('person', 'http://www.wikidata.org/entity/Q723685'), ('name', 'video on demand')]
[('person', 'http://www.wikidata.org/entity/Q22812458'), ('name', 'comedic tel

In [ ]:
# we have little knowledge of data about "The Little Mermaid"

Final query for this task

In [363]:
# person who worked on both "The Little Mermaid" and "The Lion King"
queryString = """
SELECT DISTINCT ?person ?name ?film
WHERE {
   # bind something
   wd:Q944287 ?p ?person.
   wd:Q36479 ?p ?person.
   ?person wdt:P31 wd:Q5.
   # get the label
   ?person sc:name ?name.
   
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q310292'), ('name', 'Jim Cummings')]
1


## Task 8
Identify the BGP for Academy Awards

Final query for this task

In [456]:
# wdt:P1411 -- nominated for

queryString = """
SELECT DISTINCT ?name ?p
WHERE {
   # bind something
   ?obj wdt:P1411 ?p.
   # get the label
   ?p sc:name ?name.
}
#LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'South Indian International Movie Awards'), ('p', 'http://www.wikidata.org/entity/Q7567536')]
[('name', 'Feroz Award'), ('p', 'http://www.wikidata.org/entity/Q15728786')]
[('name', 'Premio Feroz for Best Main Actor'), ('p', 'http://www.wikidata.org/entity/Q15731913')]
[('name', 'Premio Feroz for Best Supporting Actress'), ('p', 'http://www.wikidata.org/entity/Q15735989')]
[('name', 'Premio Feroz for Best Trailer'), ('p', 'http://www.wikidata.org/entity/Q15818260')]
[('name', 'Premio Feroz for Best Film Poster'), ('p', 'http://www.wikidata.org/entity/Q15818265')]
[('name', 'The Game Awards − Best Esports Coach'), ('p', 'http://www.wikidata.org/entity/Q104117261')]
[('name', 'Academy of Canadian Cinema and Television Award for Best Performance by an Actress in a Leading Role'), ('p', 'http://www.wikidata.org/entity/Q547316')]
[('name', 'Africa Movie Academy Award for Most Promising Actor'), ('p', 'http://www.wikidata.org/entity/Q17001922')]
[('name', 'Indian Telly Award

In [ ]:
# after looking up all the possible nominies, academy Academy Awards are easily found
# the main entity representing the academy awards is Q19020

# [('name', 'Academy Awards'), ('p', 'http://www.wikidata.org/entity/Q19020')]

## Task 9
For each films created by the Walt Disney Animation Studios return the number of nomination for an Academy Awards. Return only films with at least one nomination. (the result set must be triples of films IRI, label and #nomination).

In [460]:
# all the awards given during the Academy Awards ceremony are obtainable inspecting the object related to the Academy Awards entity Q19020
 
queryString = """
SELECT DISTINCT ?name ?p
WHERE {
   # bind something
   wd:Q19020 ?obj ?p.
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Academy Award for Best Picture'), ('p', 'http://www.wikidata.org/entity/Q102427')]
[('name', 'Academy Award for Best Director'), ('p', 'http://www.wikidata.org/entity/Q103360')]
[('name', 'Academy Award for Best Actress'), ('p', 'http://www.wikidata.org/entity/Q103618')]
[('name', 'Academy Award for Best Supporting Actress'), ('p', 'http://www.wikidata.org/entity/Q106301')]
[('name', 'Academy Award for Best Writing, Adapted Screenplay'), ('p', 'http://www.wikidata.org/entity/Q107258')]
[('name', 'Academy Award for Best Original Song'), ('p', 'http://www.wikidata.org/entity/Q112243')]
[('name', 'United States of America'), ('p', 'http://www.wikidata.org/entity/Q30')]
[('name', 'Academy Award for Best Writing, Original Screenplay'), ('p', 'http://www.wikidata.org/entity/Q41417')]
[('name', 'Academy Award for Best Actor'), ('p', 'http://www.wikidata.org/entity/Q103916')]
[('name', 'Academy Award for Best Foreign Language Film'), ('p', 'http://www.wikidata.org/entity/Q10

In [ ]:
#we have now a way to examine every nomitation related to a film

In [ ]:
# films directed by walt disney animation studios

queryString = """
SELECT DISTINCT ?name ?film
WHERE {
   # bind something
   ?film wdt:P272 wd:Q1047410.
   # get the label
   ?film sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('name', "The Emperor's New Groove"), ('film', 'http://www.wikidata.org/entity/Q223163')]
[('name', 'Moana'), ('film', 'http://www.wikidata.org/entity/Q18647981')]
[('name', 'Beauty and the Beast'), ('film', 'http://www.wikidata.org/entity/Q179673')]
[('name', 'Roger Rabbit short films'), ('film', 'http://www.wikidata.org/entity/Q15046569')]
[('name', 'Trail Mix-Up'), ('film', 'http://www.wikidata.org/entity/Q3362144')]
[('name', 'Who Framed Roger Rabbit'), ('film', 'http://www.wikidata.org/entity/Q379873')]
[('name', 'The Lion King'), ('film', 'http://www.wikidata.org/entity/Q36479')]
[('name', 'Sleeping Beauty'), ('film', 'http://www.wikidata.org/entity/Q215617')]
[('name', 'Bolt'), ('film', 'http://www.wikidata.org/entity/Q212792')]
[('name', 'The Little Mermaid'), ('film', 'http://www.wikidata.org/entity/Q944287')]
[('name', "Let's Stick Together"), ('film', 'http://www.wikidata.org/entity/Q3236849')]
[('name', 'Ralph Breaks the Internet'), ('film', 'http://www.wikidata.or

In [ ]:
# test -- kernel not working
# films with Disney ID and with at least one nomination

queryString = """
SELECT DISTINCT ?name ?film (COUNT (?nom) as ?nominations)
WHERE {
   # bind something
   wd:Q19020 ?obj ?nom.
   ?film wdt:P1411 ?nom;
         wdt:P6181 ?g.
   # get the label
   ?film sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Final query for this task

In [476]:
# The query returns only films with at least one nomination. (the result set are triples of films IRI, label and #nomination)

queryString = """
SELECT DISTINCT ?name ?film (COUNT(?nom) as ?nominations)
WHERE {
   # bind something
   wd:Q19020 ?obj ?nom.
   ?film wdt:P272 wd:Q1047410;
         wdt:P1411 ?nom.
   # get the label
   ?film sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Moana'), ('film', 'http://www.wikidata.org/entity/Q18647981'), ('nominations', '1')]
[('name', 'Frozen'), ('film', 'http://www.wikidata.org/entity/Q246283'), ('nominations', '1')]
[('name', 'The Lion King'), ('film', 'http://www.wikidata.org/entity/Q36479'), ('nominations', '1')]
[('name', 'Who Framed Roger Rabbit'), ('film', 'http://www.wikidata.org/entity/Q379873'), ('nominations', '1')]
[('name', 'Zootopia'), ('film', 'http://www.wikidata.org/entity/Q15270647'), ('nominations', '1')]
[('name', 'Big Hero 6'), ('film', 'http://www.wikidata.org/entity/Q13091172'), ('nominations', '1')]
[('name', 'Wreck-It Ralph'), ('film', 'http://www.wikidata.org/entity/Q28891'), ('nominations', '1')]
[('name', 'Beauty and the Beast'), ('film', 'http://www.wikidata.org/entity/Q179673'), ('nominations', '2')]
[('name', 'Bolt'), ('film', 'http://www.wikidata.org/entity/Q212792'), ('nominations', '1')]
9


In [ ]:
#once again we get a smaller result than expected, the problem with the film directed by Walt Disney studio persists, despite the lack of data about it we still manage to get some results.